In [1]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [2]:
from urllib.parse import urlparse, parse_qs
url = urlparse("http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey=13nWfyvckU06jS8j02cgKEde9gF7%2B9BZ%2BOFcpgpuu%2FnNRbR64QYGlNEzN7b%2F9D4RI3uFKkJ97YT9mf5X3FMIUA%3D%3D&numOfRows=10&pageNo=1&dataCd=ASOS&dateCd=HR&stnIds=108&schListCnt=10&endDt=20200101&endHh=01&startHh=01&startDt=20100101&dataType=JSON")
par_dic = parse_qs(url.query)
for key in par_dic:
    par_dic[key] = par_dic[key][0]
par_dic

{'serviceKey': '13nWfyvckU06jS8j02cgKEde9gF7+9BZ+OFcpgpuu/nNRbR64QYGlNEzN7b/9D4RI3uFKkJ97YT9mf5X3FMIUA==',
 'numOfRows': '10',
 'pageNo': '1',
 'dataCd': 'ASOS',
 'dateCd': 'HR',
 'stnIds': '108',
 'schListCnt': '10',
 'endDt': '20200101',
 'endHh': '01',
 'startHh': '01',
 'startDt': '20100101',
 'dataType': 'JSON'}

In [3]:
def get_hourly_data(start_date, start_hour,end_date,end_hour):
    start_date=start_date
    start_hour=start_hour
    end_date=end_date
    end_hour=end_hour
    url='http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'

    
    params={'serviceKey': '13nWfyvckU06jS8j02cgKEde9gF7+9BZ+OFcpgpuu/nNRbR64QYGlNEzN7b/9D4RI3uFKkJ97YT9mf5X3FMIUA==',
     'numOfRows': '10',
     'pageNo': '1',
     'dataCd': 'ASOS',
     'dateCd': 'HR',
     'stnIds': '108',
     'schListCnt': '10',
     'endDt': end_date,
     'endHh': end_hour,
     'startHh': start_hour,
     'startDt': start_date,
     'dataType': 'JSON'
           }

    resp = requests.get(url, params=params)
    #result= json.loads(resp.text)

    return resp.text

In [4]:
from urllib.parse import urlparse, parse_qs
url = urlparse("http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=13nWfyvckU06jS8j02cgKEde9gF7%2B9BZ%2BOFcpgpuu%2FnNRbR64QYGlNEzN7b%2F9D4RI3uFKkJ97YT9mf5X3FMIUA%3D%3D&numOfRows=10&pageNo=1&dataCd=ASOS&dateCd=DAY&startDt=20100101&endDt=20100102&stnIds=108")
par_dic = parse_qs(url.query)
for key in par_dic:
    par_dic[key] = par_dic[key][0]
par_dic

{'serviceKey': '13nWfyvckU06jS8j02cgKEde9gF7+9BZ+OFcpgpuu/nNRbR64QYGlNEzN7b/9D4RI3uFKkJ97YT9mf5X3FMIUA==',
 'numOfRows': '10',
 'pageNo': '1',
 'dataCd': 'ASOS',
 'dateCd': 'DAY',
 'startDt': '20100101',
 'endDt': '20100102',
 'stnIds': '108'}

In [5]:
def get_daily_data(start_date,end_date):
    start_date=start_date
    end_date=end_date
    url='http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'

    
    params={'serviceKey': unquote('13nWfyvckU06jS8j02cgKEde9gF7%2B9BZ%2BOFcpgpuu%2FnNRbR64QYGlNEzN7b%2F9D4RI3uFKkJ97YT9mf5X3FMIUA%3D%3D'),
            'numOfRows': '200',
            'pageNo': '1',
            'dataCd': 'ASOS',
            'dateCd': 'DAY',
            'startDt': start_date,
            'endDt': end_date,
            'stnIds': '108',
            'dataType':'JSON'
           }

    resp = requests.get(url, params=params)
    result= json.loads(resp.text)
    return result

In [6]:
def data_to_DataFrame(result):
    total_data=result['response']['body']['items']['item']
    
    result = {'날짜':[],
              '최저기온':[],
              '최고기온':[],
              '비':[],
              '강수량':[],
              '날씨':[],
             }
    for data in total_data:
        avg = (data['minTa']+data['maxTa'])/2
        result['날짜'].append(data['tm'])
        result['최저기온'].append(data['minTa'])
        result['최고기온'].append(data['maxTa'])
        avg = (data['minTa']+data['maxTa'])/2
        if data['sumRn']>0:
            result['비'].append('O')
            result['날씨'].append('비')
        else:
            if avg<10:
                result['날씨'].append('추위')
            elif avg<20:
                result['날씨'].append('쌀쌀')
            else:
                result['날씨'].append('더위')
            result['비'].append('X')
        result['강수량'].append(data['sumRn'])
        
        
    df=pd.DataFrame(result)
    
    return df

In [7]:

def append_today(data):
    url = 'https://www.weatheri.co.kr/forecast/forecast01.php'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, from_encoding='utf-8')
    attrs={'width':'615',
      'border':'0',
      'cellpadding':'1',
      'cellspacing':'1',
       'bgcolor':'#BCBFC2'
      }
    inner_table = soup.find('table',attrs=attrs)
    date = inner_table.select('td b')[0].text
    date = '2020-{0}-{1}'.format(date.split()[0][:2],date.split()[1][:2])
    high = inner_table.select('table tr td b font')[0].text
    high = high.replace('˚C','')
    low = inner_table.select('table tr td b font')[0].text
    low = low.replace('˚C','')
    sum_rain=inner_table.select('table tr td font')[-1].text
    avg = (int(low)+int(high))/2
    if sum_rain !='- mm':
        rain='O',
        sum_rain = 0
        weather = '비'
    else:
        rain='X'
        sum_rain.split()[1][:-2]
        if avg<10:
            weather = '추위'
        elif avg<20:
            weather = '쌀쌀'
        else:
            weather = '더위'
    
    new_data=pd.DataFrame({'날짜':[date],
             '최저기온':[low],
             '최고기온':[high],
             '비':[rain],
             '강수량':[sum_rain],
             '날씨':[weather]})
    
    data = data.append(new_data,ignore_index=True)
    print(data)
    return data

In [8]:
result=get_daily_data('20200501','20200609')
output=data_to_DataFrame(result)
d=append_today(output)

            날짜  최저기온  최고기온     비   강수량  날씨
0   2020-05-01  16.4  26.2     X   0.0  더위
1   2020-05-02    18  23.9     X   0.0  더위
2   2020-05-03    17  27.4     X   0.0  더위
3   2020-05-04  14.8  25.3     X   0.0  더위
4   2020-05-05  13.1  19.3     X   0.0  쌀쌀
5   2020-05-06  11.1  27.6     X   0.0  쌀쌀
6   2020-05-07  14.9  26.5     X   0.0  더위
7   2020-05-08  14.3  27.3     O   2.3   비
8   2020-05-09  12.7  16.8     O  24.4   비
9   2020-05-10  12.8  16.6     O   1.3   비
10  2020-05-11  12.4    22     O   3.4   비
11  2020-05-12  10.5  18.9     O   0.7   비
12  2020-05-13  10.2  21.8     X   0.0  쌀쌀
13  2020-05-14  12.4  25.4     X   0.0  쌀쌀
14  2020-05-15  14.1  20.2     O  12.0   비
15  2020-05-16    15  20.3     O   2.5   비
16  2020-05-17  14.6  24.9     X   0.0  쌀쌀
17  2020-05-18  13.2  26.8     O  29.8   비
18  2020-05-19  10.1  17.1     O  19.0   비
19  2020-05-20   9.2  20.1     X   0.0  쌀쌀
20  2020-05-21  10.9  22.9     X   0.0  쌀쌀
21  2020-05-22  14.8  22.7     X   0.0  쌀쌀
22  2020-05

### 시작

In [9]:
def enter_weather(weather):
    header = {'User-Agent': 'Mozilla/5.0'}    # melon 크롤링 시 header값 필요
    base_url = 'https://www.melon.com/dj/djfinder/djfinder_inform.htm'

    params = {
        'djSearchKeyword':weather
    }
    resp = requests.get(base_url,headers=header,params=params)
    soup = BeautifulSoup(resp.text)
    #print(resp)
    #soup
    # playlist의 seq를 뽑아 리스트에 넣기
    ply_list_dj = soup.select('div.service_list_play.d_djcol_list li')
    #print(ply_list_dj)
    seq = []
    for li in ply_list_dj:
        a_tag = li.find('a')
        #print(a_tag)
        par = a_tag.get('href')
        #print(par)
        seq.append(int(par[-12:-3]))
    return seq

In [10]:
def title_and_singer(seq):
    titles = []         # 각 플레이리스트별로 리스트 초기화
    singers = []
    song = []
    likes = []
    for i in range(len(seq)):
        new_url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm'
        new_params = {
        'plylstSeq': seq[i]
        }
        hdr={'User-Agent': 'Mozilla/5.0'}
        new_resp = requests.get(new_url,headers=hdr,params=new_params)
        new_soup = BeautifulSoup(new_resp.text)

        song_list = new_soup.find_all('div',class_='wrap_song_info')
        #     print(song_list)

        for wrap in song_list:
            title = wrap.find('div',class_='ellipsis rank01')
            singer = wrap.find('div',class_='ellipsis rank02')
            if singer != None:
                leng = round(len(singer.text)/2)
                singers.append(singer.text[:leng].strip())
        #     print(singers)
            if title != None:
                titles.append(title.text.strip())
        # print(titles,len(titles))    
        # print(singers,len(singers))
#         print('\n')
        for ts in range(len(titles)):
            song.append((titles[ts],singers[ts]))
        #     song = list(zip(titles,singers))
        likes = likes_num(seq[i])
        for tsl in range(len(likes)):
            if song[tsl] in info:
                info[song[tsl]] = likes[tsl]+100
            else:
                info[song[tsl]] = likes[tsl]
    return info

        #     print('\n\n')

In [11]:
def likes_num(sq):
    likes = []
    new_url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm'
    new_params = {
        'plylstSeq': sq
    }                                       # 각 플레이리스트 url 생성
    header = {'User-Agent': 'Mozilla/5.0'}
    new_resp = requests.get(new_url,headers = header,params=new_params)
    new_soup = BeautifulSoup(new_resp.text)
    song_in_plylist = new_soup.select('div.wrap button')

    # print(song_in_plylist)
    song_no = []
    for button in song_in_plylist:
        no = button.get('data-song-no')
        if no != None:
            song_no.append(no)
    #print(song_no)
    song_no_select = song_no[0]
    for s in range(1,len(song_no)):
        song_no_select = song_no_select +','+str(song_no[s])
    ply_url = 'https://www.melon.com/commonlike/getSongLike.json'
    # print(song_no_select)
    ply_params = {
        "contsIds": song_no_select
        }
    referer = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq='+str(sq)
    # print(referer)

    hd = {
        'User-Agent': 'Mozilla/5.0',
        'referer':referer
        }
    ply_resp = requests.get(ply_url,headers=hd,params=ply_params)
    # ply_resp
    result= json.loads(ply_resp.text)
    # result
    extract = result['contsLike']
    # print(extract)
    for i in range(len(song_no)):
        likes.append(extract[i]['SUMMCNT'])
    return likes
    # likes

# likes_num()

In [12]:
result_={
        '날짜':[],
        '노래':[],
        '가수':[],
        '좋아요 수':[]
    }
for k in range(len(d['날씨'])):
    info = {}
    date = d['날짜'][k]
    curr_seq = enter_weather(d['날씨'][k])
    print(d['날짜'][k],d['날씨'][k])
#     print(curr_seq)
    result_info = title_and_singer(curr_seq)
    result_info = sorted(result_info.items(), key=lambda x: x[1], reverse=True)
    for j in range(5):
        result_['날짜'].append(date),
        result_['노래'].append(result_info[j][0][0]),
        result_['가수'].append(result_info[j][0][1]),
        result_['좋아요 수'].append(result_info[j][1])
print(result_)

{'날짜': ['2020-05-01', '2020-05-01', '2020-05-01', '2020-05-01', '2020-05-01', '2020-05-02', '2020-05-02', '2020-05-02', '2020-05-02', '2020-05-02', '2020-05-03', '2020-05-03', '2020-05-03', '2020-05-03', '2020-05-03', '2020-05-04', '2020-05-04', '2020-05-04', '2020-05-04', '2020-05-04', '2020-05-05', '2020-05-05', '2020-05-05', '2020-05-05', '2020-05-05', '2020-05-06', '2020-05-06', '2020-05-06', '2020-05-06', '2020-05-06', '2020-05-07', '2020-05-07', '2020-05-07', '2020-05-07', '2020-05-07', '2020-05-08', '2020-05-08', '2020-05-08', '2020-05-08', '2020-05-08', '2020-05-09', '2020-05-09', '2020-05-09', '2020-05-09', '2020-05-09', '2020-05-10', '2020-05-10', '2020-05-10', '2020-05-10', '2020-05-10', '2020-05-11', '2020-05-11', '2020-05-11', '2020-05-11', '2020-05-11', '2020-05-12', '2020-05-12', '2020-05-12', '2020-05-12', '2020-05-12', '2020-05-13', '2020-05-13', '2020-05-13', '2020-05-13', '2020-05-13', '2020-05-14', '2020-05-14', '2020-05-14', '2020-05-14', '2020-05-14', '2020-05-15'

In [13]:
df = pd.DataFrame(result_)
df.to_csv('weather_melon.csv')

In [14]:
df

,날짜,노래,가수,좋아요 수
0,2020-05-01,Toxic,"Glaceo, Cristina Lizzul",481718
1,2020-05-01,Wasted Summer,"teamwork, Loote, John K",283256
2,2020-05-01,Lie To Me,"Marphil, Yann Muller, Sylow",182396
3,2020-05-01,Freedom,"Kygo, Zak Abel",181091
4,2020-05-01,빨간 맛 (Red Flavor),Red Velvet (레드벨벳),95193
...,...,...,...,...
200,2020-06-10,사랑은 언제나 목마르다,유미,4836
201,2020-06-10,내 입술... 따뜻한 커피처럼,샵,3972
202,2020-06-10,오늘 헤어졌어요,윤하 (YOUNHA),2428
203,2020-06-10,희재,성시경,1391
